# UDA-Hub: Multi-Agent Customer Support System

This notebook demonstrates the LangGraph-based multi-agent system for CultPass customer support.

**Prerequisites:**
- Run `01_external_db_setup.py` (or notebook) first
- Run `02_core_db_setup.py` (or notebook) first
- Have `.env` file with `OPENAI_API_KEY`

## 1. Setup & Imports

In [ ]:
# Load environment variables FIRST (before any other imports)
from dotenv import load_dotenv
load_dotenv()

print("✅ Environment loaded")

In [ ]:
# Import the orchestrator and message types
from agentic.workflow import orchestrator
from langchain_core.messages import HumanMessage, AIMessage

print("✅ Orchestrator loaded")
print("\nWorkflow nodes:", list(orchestrator.get_graph().nodes.keys()))

## 2. Helper Function

Simple function to send a message and get a response.

In [ ]:
def ask(message: str, thread_id: str = "notebook-1") -> str:
    """
    Send a message to the support agent and get a response.
    
    Args:
        message: User's question or issue
        thread_id: Session ID for memory (same ID = same conversation)
    
    Returns:
        Agent's response
    """
    result = orchestrator.invoke(
        input={"messages": [HumanMessage(content=message)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Get the last AI message
    response = result["messages"][-1].content
    
    # Also return metadata for debugging
    return {
        "response": response,
        "category": result.get("category", "unknown"),
        "escalated": result.get("needs_escalation", False)
    }

print("✅ Helper function ready")

## 3. Test: Simple Questions (Resolver Path)

These queries should be handled by the Resolver Agent using RAG.

In [ ]:
# Test: Login issue
result = ask("I can't log into my account")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# Test: Subscription question
result = ask("What's included in my subscription?")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# Test: Reservation question
result = ask("How do I cancel my reservation?")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# Test: QR code issue (tests semantic search)
result = ask("My QR code won't scan at the venue")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

## 4. Test: Escalation Path

These queries should trigger escalation to human support.

In [ ]:
# Test: Blocked account (auto-escalates)
result = ask("My account is blocked and I don't know why")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

In [ ]:
# Test: Explicit human request
result = ask("I want to speak to a human agent please")

print(f"Category: {result['category']}")
print(f"Escalated: {result['escalated']}")
print(f"\nResponse:\n{result['response']}")

## 5. Test: RAG Semantic Search

Test that semantic search finds relevant articles even without exact keyword matches.

In [ ]:
# Direct test of RAG search
from agentic.tools.rag_tools import search_knowledge_base

# This should find login articles even though we don't say "login" or "password"
result = search_knowledge_base.invoke("I can't get into my account")
print("Query: 'I can't get into my account'")
print("="*50)
print(result[:1500])  # First 1500 chars

In [ ]:
# This should find refund articles
result = search_knowledge_base.invoke("I want my money back")
print("Query: 'I want my money back'")
print("="*50)
print(result[:1500])

## 6. Test: Conversation Memory

Test that the agent remembers context within a session.

In [ ]:
# Start a new conversation thread
thread = "memory-test-1"

# First message
result1 = ask("I'm having trouble with my premium subscription", thread_id=thread)
print("User: I'm having trouble with my premium subscription")
print(f"Agent: {result1['response'][:300]}...")
print()

In [ ]:
# Follow-up in same thread (should remember context)
result2 = ask("What benefits do I get with it?", thread_id=thread)
print("User: What benefits do I get with it?")
print(f"Agent: {result2['response']}")

## 7. Test: Individual Agents

Test each agent in isolation.

In [ ]:
# Test Classifier Agent directly
from agentic.agents.classifier import classify_ticket
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

test_messages = [
    "I forgot my password",
    "How much does premium cost?",
    "My account is suspended",
    "The app keeps crashing",
    "I want a refund"
]

print("Classifier Agent Test Results:")
print("=" * 50)
for msg in test_messages:
    result = classify_ticket(msg, model)
    print(f"'{msg}'")
    print(f"  → Category: {result['category']}, Escalate: {result['needs_escalation']}")
    print()

In [ ]:
# Test Resolver Agent directly
from agentic.agents.resolver import generate_response

response = generate_response(
    user_message="How do I book an experience?",
    category="reservation",
    knowledge_context="To reserve an experience, open the app, browse events, and tap Reserve.",
    user_context="",
    model=model
)

print("Resolver Agent Response:")
print("=" * 50)
print(response)

In [ ]:
# Test Escalation Agent directly
from agentic.agents.escalation import generate_escalation_response

response = generate_escalation_response(
    user_message="My account got blocked for no reason!",
    reason="account_blocked",
    user_context="User: John Doe, Email: john@example.com",
    model=model
)

print("Escalation Agent Response:")
print("=" * 50)
print(response)

## 8. Interactive Chat

Have a conversation with the support agent.

In [ ]:
# Interactive chat loop
# Run this cell and type messages. Type 'quit' to stop.

print("💬 Chat with UDA-Hub Support Agent")
print("Type 'quit' to exit")
print("=" * 40)

thread_id = "interactive-chat"

while True:
    user_input = input("\nYou: ").strip()
    
    if user_input.lower() in ['quit', 'exit', 'q']:
        print("\nGoodbye!")
        break
    
    if not user_input:
        continue
    
    result = ask(user_input, thread_id=thread_id)
    print(f"\nAgent: {result['response']}")
    print(f"\n[Category: {result['category']}, Escalated: {result['escalated']}]")

## 9. Visualize the Workflow Graph

In [ ]:
# Display the workflow graph (requires graphviz)
try:
    from IPython.display import Image, display
    
    # Get the graph image
    graph_image = orchestrator.get_graph().draw_mermaid_png()
    display(Image(graph_image))
except Exception as e:
    print(f"Could not render graph: {e}")
    print("\nWorkflow structure:")
    print("  START → classify → [resolve OR escalate] → END")

## Summary

This notebook demonstrated:

1. **Classifier Agent** - Categorizes tickets into: login_access, subscription, billing, reservation, technical, account_blocked, general_inquiry, escalation

2. **Resolver Agent** - Uses RAG (Chroma + OpenAI embeddings) to find relevant KB articles and generate responses

3. **Escalation Agent** - Handles blocked accounts and human requests by routing to support team

4. **RAG System** - Semantic search finds articles even without exact keyword matches

5. **Memory** - Conversation context maintained via thread_id

### Architecture

```
User Message
     ↓
┌─────────────┐
│  CLASSIFY   │
└─────────────┘
     ↓
   Router
     ↓
┌─────────────┐         ┌─────────────┐
│   RESOLVE   │   OR    │  ESCALATE   │
│ (RAG + LLM) │         │ (to human)  │
└─────────────┘         └─────────────┘
```